### Task

- group the icd code by patient 
- figure out the top 10 diagnoses based on the # of hospitalizations 

### important files:

1. diagnoses_icd.csv (~4500 rows) -- Each row links a patient (subject_id) and a hospitalization (hadm_id)
   to an ICD code. Columns: subject_id, hadm_id, seq_num, icd_code, icd_version

2. d_icd_diagnoses.csv -- The dictionary/lookup table that maps each icd_code to a human-readable long_title
   (e.g., code 4139 = "Angina pectoris, unspecified")

3. admissions.csv.gz -- One row per hospitalization, with subject_id and hadm_id

### Logic
- Each unique hadm_id in diagnoses_icd.csv represents one hospitalization
- Each hospitalization has multiple ICD codes (diagnoses) listed
- "Group by patient" means looking at which ICD codes each subject_id has
- "Top 10 diagnoses by number of hospitalizations" = count how many distinct hadm_id values each icd_code
    appears in, then rank

In [3]:
# Cell 1: Just setup -- imports pandas and defines where the data lives

# looks good to me yo !

import pandas as pd 


# path to the hosp folder 
DATA_DIR = "physionet.org/files/mimic-iv-demo/2.2/hosp"


In [4]:
# Cell 2: Loads both CSVs. The dtype={"icd_code": str} is important because some ICD codes have leading
# zeros (like 0090) that would get stripped if pandas reads them as numbers

# diagnoses_icd.csv: each row is one diagnoses for one hospitalization 
# columns: subject_id (patient), hadm_id (hospitalization), seq_num (priority),
           # icd_code (diagnosis code), icd_version (9 or 10)

diagnoses = pd.read_csv(f"{DATA_DIR}/diagnoses_icd.csv", dtype={"icd_code": str})

#d_icd_diagnoses.csv lookup table mapping icd code -> human-readable name
# columns: icd_code, icd_version, long_title 
icd_lookup = pd.read_csv(f"{DATA_DIR}/d_icd_diagnoses.csv", dtype={"icd_code": str})

print(f"Diagnoses rows: {len(diagnoses)}")
print(f"ICD lookup entries: {len(icd_lookup)}")
diagnoses.head()

Diagnoses rows: 4506
ICD lookup entries: 109775


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10035185,22580999,3,4139,9
1,10035185,22580999,10,V707,9
2,10035185,22580999,1,41401,9
3,10035185,22580999,9,3899,9
4,10035185,22580999,11,V8532,9


In [5]:
# Cell 3: Groups by subject_id to show all ICD codes per patient (this is the "group by patient" part of
#  your instructions)
  
  # Group all diagnoses by patient (subject_id)
  # This gives us every ICD code each patient was ever diagnosed with
patient_diagnoses = (
      diagnoses
      .groupby("subject_id")["icd_code"]
      .apply(list)
      .reset_index()
      .rename(columns={"icd_code": "all_icd_codes"})
)


# How many unique diagnoses per patient
patient_diagnoses["num_diagnoses"] = patient_diagnoses["all_icd_codes"].apply(len)

print(f"Total patients: {len(patient_diagnoses)}")
patient_diagnoses.head()  

Total patients: 100


,subject_id,all_icd_codes,num_diagnoses
0,10000032,"[2761, V08, 2875, 3051, 5715, 496, 07071, 7895...",39
1,10001217,"[V168, V161, 04109, 3485, 4019, 3484, 3240, 34...",17
2,10001725,"[49390, 7291, 6185, 56400, 5641, E9352, 6186, ...",18
3,10002428,"[4019, 29281, 4240, 00845, 1122, 7102, 73300, ...",114
4,10002495,"[R310, N179, Z7901, Z86718, I214, A047, Z23, K...",26


In [6]:
# Cell 4: The main result -- counts distinct hospitalizations per ICD code, sorts descending, takes top 10,
#  and joins with the lookup table so you see actual diagnosis names instead of just codes
  
  # Count how many DISTINCT hospitalizations (hadm_id) each ICD code appears in.
  # We use nunique() because a code could theoretically appear multiple times
  # in the same admission, but we want to count each hospitalization once.
top_diagnoses = (
      diagnoses
      .groupby(["icd_code", "icd_version"])["hadm_id"]
      .nunique()
      .reset_index()
      .rename(columns={"hadm_id": "num_hospitalizations"})
      .sort_values("num_hospitalizations", ascending=False)
      .head(10)
)

# Merge with the lookup table to get human-readable names
top_diagnoses = top_diagnoses.merge(icd_lookup, on=["icd_code", "icd_version"], how="left")

# Reorder columns for readability
top_diagnoses = top_diagnoses[["icd_code", "icd_version", "long_title", "num_hospitalizations"]]

top_diagnoses

,icd_code,icd_version,long_title,num_hospitalizations
0,4019,9,Unspecified essential hypertension,68
1,E785,10,"Hyperlipidemia, unspecified",57
2,2724,9,Other and unspecified hyperlipidemia,55
3,E039,10,"Hypothyroidism, unspecified",47
4,Z794,10,Long term (current) use of insulin,37
5,Z87891,10,Personal history of nicotine dependence,35
6,42731,9,Atrial fibrillation,34
7,25000,9,Diabetes mellitus without mention of complicat...,33
8,I2510,10,Atherosclerotic heart disease of native corona...,33
9,I10,10,Essential (primary) hypertension,32
